# EfficientNetB0


## I. Setup


In [1]:
import os
import sys

# os.chdir('/Volumes/SD Card/cleaned')
# os.getcwd()

sys.path.append('..')

In [2]:
def listdir(path: str):
    return os.listdir(path)

In [3]:
dataset_path = '/Volumes/SD Card/cleaned'

# cosonant_path = f'{dataset_path}/cosonant'
# independent_vowel_path = f'{dataset_path}/independent_vowel'
# stack_consonant_vowel_path = f'{dataset_path}/stack_consonant'
# sub_vowel_path = f'{dataset_path}/sub_vowel'
# vowel_path = f'{dataset_path}/vowel'

# images_count = 0
# for inner1 in listdir(f'{dataset_path}'):
#     for inner2 in listdir(f'{dataset_path}/{inner1}'):
#         try:
#             images_count = images_count + len(listdir(f'{dataset_path}/{inner1}/{inner2}'))
#         except NotADirectoryError as e:
#             # print(f'ERROR: {e}')
#             continue

# images_count

33168

In [4]:
# only test consonant for now
cosonant_path = '/Volumes/SD Card/cleaned_hand'

## II. Prepare dataset


In [5]:
import pandas as pd
import random


In [6]:
df = pd.DataFrame(columns=["label", "path"])

In [7]:
for cosonant in listdir(cosonant_path):
    variants_path = f'{cosonant_path}/{cosonant}'
    variants = listdir(f'{cosonant_path}/{cosonant}')

    # for variant in variants:
    for variant in random.sample(variants, min(10, len(variants))):
    # for variant in [variants[0], variants[1], variants[2], variants[3]]:
        vpath = f'{variants_path}/{variant}'
        df.loc[len(df.index)] = [cosonant, vpath]

In [8]:
df.shape

(321, 2)

In [9]:
df.head(5)

,label,path
0,vo,/Volumes/SD Card/cleaned_hand/vo/493.jpg
1,vo,/Volumes/SD Card/cleaned_hand/vo/475.jpg
2,vo,/Volumes/SD Card/cleaned_hand/vo/462.jpg
3,vo,/Volumes/SD Card/cleaned_hand/vo/328.jpg
4,vo,/Volumes/SD Card/cleaned_hand/vo/170.jpg


In [10]:
df["label"].unique()


array(['vo', 'cha', 'cho', 'chha', 'tho1', 'chho', 'mo', 'da', 'do', 'no',
       'na', 'nho', 'tha1', 'ro', 'yo', 'sa', 'po', 'kha', 'kho', 'ba',
       'ha', 'tho2', 'ka', 'ko', 'ngo', 'or', 'tha2', 'lo', 'la', 'ta',
       'to', 'pho', 'pha'], dtype=object)

## III. Preprocessing


### a. Image


In [11]:
import tensorflow as tf
import cv2
import numpy as np

images = []
labels = []
error_count = 0


def resize_image(image_path):
    image = cv2.imread(image_path)
    image = tf.image.resize(image, size=(224, 224))
    image = tf.cast(image, dtype=tf.float32)
    return image


for item in np.array(df):
    label = item[0]
    path = item[1]

    try:
        image = resize_image(path)
        images.append(image)
        labels.append(label)
    except ValueError as e:
        error_count = error_count + 1

error_count, len(images), len(labels)


2022-11-18 15:09:16.954827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 15:09:21.472149: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(0, 321, 321)

In [12]:
images = np.array(images)
labels = np.array(labels)


### b. Label


In [13]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(labels)
Y = y.reshape(-1, 1)

Y.shape


(321, 1)

### c. Split testing & training data


In [14]:
from sklearn.model_selection import train_test_split


In [15]:
train_x, test_x, train_Y, test_Y = train_test_split(
    images,
    Y,
    test_size=0.2,
    random_state=10
)


In [16]:
train_x.shape, test_x.shape, train_Y.shape, test_Y.shape


((256, 224, 224, 3), (65, 224, 224, 3), (256, 1), (65, 1))

## IV. Train


In [17]:
NUM_CLASSES = len(labels)
IMG_SIZE = 32

size = (IMG_SIZE, IMG_SIZE)


In [18]:
inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
outputs = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=True,
    weights=None,
    classes=NUM_CLASSES
)(inputs)

model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="sgd",
    loss="mse",
    metrics=["accuracy"]
)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, 321)              4460772   
                                                                 
Total params: 4,460,772
Trainable params: 4,418,749
Non-trainable params: 42,023
_________________________________________________________________


In [19]:
from datetime import datetime

In [20]:
epochs = 30

started_at = datetime.now().time()
hist = model.fit(train_x, train_Y, epochs=epochs, verbose=2)
trained_at = datetime.now().time()

hist.history

Epoch 1/30
8/8 - 49s - loss: 336.1956 - accuracy: 0.0039 - 49s/epoch - 6s/step
Epoch 2/30
8/8 - 37s - loss: 336.1957 - accuracy: 0.0039 - 37s/epoch - 5s/step
Epoch 3/30
8/8 - 34s - loss: 336.1957 - accuracy: 0.0078 - 34s/epoch - 4s/step
Epoch 4/30
8/8 - 34s - loss: 336.1956 - accuracy: 0.0000e+00 - 34s/epoch - 4s/step
Epoch 5/30
8/8 - 33s - loss: 336.1957 - accuracy: 0.0000e+00 - 33s/epoch - 4s/step
Epoch 6/30
8/8 - 37s - loss: 336.1957 - accuracy: 0.0000e+00 - 37s/epoch - 5s/step
Epoch 7/30
8/8 - 34s - loss: 336.1956 - accuracy: 0.0039 - 34s/epoch - 4s/step
Epoch 8/30
8/8 - 34s - loss: 336.1957 - accuracy: 0.0000e+00 - 34s/epoch - 4s/step
Epoch 9/30
8/8 - 34s - loss: 336.1957 - accuracy: 0.0039 - 34s/epoch - 4s/step
Epoch 10/30
8/8 - 36s - loss: 336.1957 - accuracy: 0.0039 - 36s/epoch - 5s/step
Epoch 11/30
8/8 - 33s - loss: 336.1956 - accuracy: 0.0000e+00 - 33s/epoch - 4s/step
Epoch 12/30
8/8 - 36s - loss: 336.1956 - accuracy: 0.0000e+00 - 36s/epoch - 4s/step
Epoch 13/30
8/8 - 35s - l

{'loss': [336.1956481933594,
  336.1956787109375,
  336.1956787109375,
  336.1956481933594,
  336.1956787109375,
  336.1956787109375,
  336.19561767578125,
  336.1956787109375,
  336.1956787109375,
  336.1956787109375,
  336.1956481933594,
  336.1956481933594,
  336.1956481933594,
  336.1956787109375,
  336.1956481933594,
  336.1956481933594,
  336.1956787109375,
  336.1956481933594,
  336.1956787109375,
  336.1956787109375,
  336.1956787109375,
  336.1956787109375,
  336.1956787109375,
  336.1956481933594,
  336.1956787109375,
  336.1956481933594,
  336.1956481933594,
  336.1956787109375,
  336.1956787109375,
  336.1956481933594],
 'accuracy': [0.00390625,
  0.00390625,
  0.0078125,
  0.0,
  0.0,
  0.0,
  0.00390625,
  0.0,
  0.00390625,
  0.00390625,
  0.0,
  0.0,
  0.00390625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.00390625,
  0.0078125,
  0.0,
  0.0,
  0.00390625,
  0.00390625,
  0.00390625,
  0.0,
  0.0,
  0.0,
  0.0,
  0.00390625,
  0.00390625]}

In [21]:
started_at, trained_at

(datetime.time(15, 9, 27, 873863), datetime.time(15, 26, 46, 908896))

In [22]:
prediction = model.evaluate(test_x, test_Y)

print("Loss = " + str(prediction[0]))
print("Test Accuracy = " + str(prediction[1]))

3/3 [==============================] - 4s 438ms/step - loss: 348.8834 - accuracy: 0.0000e+00
Loss = 348.8833923339844
Test Accuracy = 0.0


## V. Save Model

In [23]:
model.save(f'../models/efficient_net_b0-{trained_at}.h5')